## Projet 10: Application de recommandation de contenu
### Partie 2: Modèlisation 
Dans ce notebook on vas procéder à effectuer la modélisation des nos données pour mettre en place une systeme de recommandation du contenu.
Nous allons utiliser deux approches principales:
- Content-based filtering
- Collaborative-based filtering

Nous allons nous appuyer sur la libraries [Surpise](https://surpriselib.com/) pour mettre en place nos modèles 

### 1. Import 

#### 1.1 Import des libraries

In [4]:
import os
import pickle
import pandas as pd


from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

from math import floor
import numpy as np

import surprise
from surprise import SVD, Dataset, Reader, KNNBasic, KNNWithMeans
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from heapq import nlargest

from surprise import accuracy
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVDpp
from collections import defaultdict
from heapq import nlargest


#### 1.2 Import des données

Maintenant nous allons procéder aves l'importations des libraries que nous allons utiliser pour la modèlisation et puis on vas importer les fichiers des données que nous allons utiliser comme base pour les modèles.

#### 1.2.1 Définition des chemins

In [5]:
data_path = "../data/raw/globocom/"
clicks_path= "../data/raw/globocom/clicks/"
model_path = "../model/"

#### 1.2.2 Import fichier avec metadonnées des articles

In [6]:
articles_df = pd.read_csv(data_path + 'articles_metadata.csv')
articles_df.drop(columns=['created_at_ts'], inplace=True)
articles_df.head()

,article_id,category_id,publisher_id,words_count
0,0,0,0,168
1,1,1,0,189
2,2,1,0,250
3,3,1,0,230
4,4,1,0,162


#### 1.2.3 Import fichiers avec embeddings des articles

In [7]:
# Ouvrir le fichiers pickle et afficher les 5 premiers lignes
with open(data_path + 'articles_embeddings.pickle', 'rb') as f:
    data = pickle.load(f)

embeddings_df = pd.DataFrame(data)
embeddings_df.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,0.165183,...,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,0.398246,...,-0.487843,0.823124,0.412688,-0.338654,0.320787,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,-0.242004,...,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284
3,-0.740843,-0.975749,0.391698,0.641738,-0.268645,0.191745,-0.825593,-0.710591,-0.040099,-0.110514,...,0.271535,0.036040,0.480029,-0.763173,0.022627,0.565165,-0.910286,-0.537838,0.243541,-0.885329
4,-0.279052,-0.972315,0.685374,0.113056,0.238315,0.271913,-0.568816,0.341194,-0.600554,-0.125644,...,0.238286,0.809268,0.427521,-0.615932,-0.503697,0.614450,-0.917760,-0.424061,0.185484,-0.580292


In [8]:
pca = PCA(n_components=10)
pca.fit(embeddings_df)
embeddings_df_pca = pca.transform(embeddings_df)

In [9]:
embeddings_df_pca = pd.DataFrame(embeddings_df_pca, columns=["emb_" + str(i) for i in range(embeddings_df_pca.shape[1])])
embeddings_df_pca.to_pickle('../data/processed/embeddings_df_pca.pkl')
embeddings_df_pca.head()

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9
0,-2.176782,1.316913,-1.029048,0.901908,-1.809542,-2.064713,-1.221915,-0.024441,0.927261,-0.669806
1,-1.735174,-0.489893,3.268562,0.087855,1.473059,-0.932711,1.841631,-0.881798,-0.207201,0.816809
2,-0.912688,2.089339,1.865869,-1.202519,2.530600,-0.521970,0.224352,1.479935,-0.191900,1.356800
3,1.096567,-0.212955,4.183517,-0.649575,-0.130867,1.126555,1.063997,-0.662875,0.348143,1.463898
4,0.193785,0.263949,1.896583,-1.834345,1.270377,-1.723296,0.329006,0.283794,-0.659809,1.223740


#### 1.2.4 Import fichier avec les interactions des utilisateurs

In [10]:
def get_all_files_clicks(path):
    clicks_df = pd.DataFrame()
    for file in os.listdir(path):
        df = pd.read_csv(path + file)
        clicks_df = pd.concat([clicks_df, df], axis=0)

    return clicks_df

In [11]:
clicks_df = get_all_files_clicks(clicks_path)

In [12]:
clicks_df['click_timestamp'] = pd.to_datetime(clicks_df['click_timestamp'], unit='ms')
clicks_df['session_start'] = pd.to_datetime(clicks_df['session_start'], unit='ms')
clicks_df.to_pickle('../data/processed/clicks_df.pkl')
clicks_df.head()

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,93863,1507865792177843,2017-10-13 03:36:32,2,96210,2017-10-13 03:37:12.925,4,3,2,1,21,2
1,93863,1507865792177843,2017-10-13 03:36:32,2,158094,2017-10-13 03:37:42.925,4,3,2,1,21,2
2,294036,1507865795185844,2017-10-13 03:36:35,2,20691,2017-10-13 03:36:59.095,4,3,20,1,9,2
3,294036,1507865795185844,2017-10-13 03:36:35,2,96210,2017-10-13 03:37:29.095,4,3,20,1,9,2
4,77136,1507865796257845,2017-10-13 03:36:36,2,336245,2017-10-13 03:42:13.178,4,3,2,1,25,2


### 2. Content-based Filtering

Le **Content-based Filtering** est une méthode de recommandation qui utilise des informations détaillées sur les éléments pour recommander d'autres éléments similaires. Par exemple, dans un système de recommandation de films, le filtrage basé sur le contenu pourrait utiliser des informations telles que le genre du film, le réalisateur, les acteurs, etc.

**Principe**
L'idée est que si un utilisateur a aimé un certain élément dans le passé, il est probable qu'il aimera à nouveau des éléments similaires à l'avenir. Par conséquent, le système recommande des éléments qui sont similaires aux éléments que l'utilisateur a aimés précédemment.

**Calcul de la similarité**
La similarité entre les éléments est généralement calculée en utilisant des techniques telles que la similarité cosinus ou la distance euclidienne. Les éléments qui sont les plus similaires à ceux que l'utilisateur a aimés sont recommandés.

**Note importante**
Il est important de noter que le filtrage basé sur le contenu ne tient pas compte des opinions d'autres utilisateurs. Il se concentre uniquement sur les préférences de l'utilisateur actuel.

### Étapes du Code 

1. **Préparation des Données** :
    - Les colonnes `user_id` et `click_article_id` sont converties en type entier.
    - Les articles lus par l'utilisateur cible sont identifiés.
    - Si l'utilisateur n'a lu aucun article, les articles les plus populaires sont recommandés.

2. **Filtrage Basé sur le Contenu** :
    - Les embeddings des articles lus par l'utilisateur sont obtenus.
    - Les articles lus par l'utilisateur sont retirés de la liste des articles.
    - La similarité cosinus entre les articles lus par l'utilisateur et les autres articles est calculée.
    - Les articles les plus similaires aux articles lus par l'utilisateur sont recommandés.
    - La similarité des articles recommandés est mise à zéro pour éviter de les recommander à nouveau.

3. **Vérification des Résultats** :
    - Le nombre d'articles recommandés qui ont déjà été lus par l'utilisateur est calculé et affiché.
    - Les recommandations sont retournées.

In [13]:
def recommend_articles(articles, clicks, user_id, n=5):
    # Convert user_id and click_article_id to integer type
    clicks['user_id'] = clicks['user_id'].astype(int)
    clicks['click_article_id'] = clicks['click_article_id'].astype(int)
    articles.index = articles.index.astype(int)
    
    # Get the articles read by the user
    articles_read = clicks[clicks['user_id'] == int(user_id)]['click_article_id'].tolist()
    print(f"Articles read by user {user_id}: {articles_read}")

    # If the user hasn't read any articles, recommend the most popular ones
    if len(articles_read) == 0:
        most_popular_articles = clicks['click_article_id'].value_counts().index.tolist()
        print(f"User {user_id} has not read any articles. Recommending most popular articles: {most_popular_articles[:n]}")
        return most_popular_articles[:n]

    # Get the embeddings of the articles read by the user
    articles_read_embedding = articles.loc[articles_read]
    print(f"Number of articles read by user {user_id}: {len(articles_read)}")

    # Remove the articles read by the user from the list of articles
    articles = articles.drop(articles_read)
    print(f"Remaining articles after removing articles read by user {user_id}: {len(articles)}")

    # Calculate the cosine similarity between the articles read by the user and the other articles
    matrix = cosine_similarity(articles_read_embedding, articles)

    recommendations = []

    # Recommend the articles most similar to the articles read by the user
    for i in range(n):
        coord_x = floor(np.argmax(matrix)/matrix.shape[1])
        coord_y = np.argmax(matrix)%matrix.shape[1]

        recommendations.append(int(articles.index[coord_y]))

        # Set the similarity of the recommended article to 0
        matrix[coord_x][coord_y] = 0

    # Print the number of recommended articles that have already been read by the user
    already_read = len(set(recommendations) & set(articles_read))
    print(f"Number of recommended articles that have already been read by user {user_id}: {already_read}")

    return recommendations

In [14]:
user_id = 7723

In [15]:
recommend = recommend_articles(embeddings_df_pca, clicks_df, user_id, 10)
print(f"recommended articles: {recommend}")

Articles read by user 7723: [214455, 9308, 9649, 129799, 141548, 336220, 353673, 337192, 84763, 107179, 199197, 271400, 84835, 338339, 60252, 303565, 31520, 36685, 36609, 163505, 123434, 141050, 313504, 272660, 72618, 72646, 140445, 277491, 226648, 57740, 128551, 140324, 198659, 166581, 156560, 282964, 225124, 277491, 128707]
Number of articles read by user 7723: 39
Remaining articles after removing articles read by user 7723: 364009
Number of recommended articles that have already been read by user 7723: 0
recommended articles: [141742, 128643, 127781, 139329, 85765, 139598, 89224, 139886, 34677, 128120]


In [16]:
recommend = recommend_articles(embeddings_df, clicks_df, user_id, 10)
print(f"recommended articles: {recommend}")

Articles read by user 7723: [214455, 9308, 9649, 129799, 141548, 336220, 353673, 337192, 84763, 107179, 199197, 271400, 84835, 338339, 60252, 303565, 31520, 36685, 36609, 163505, 123434, 141050, 313504, 272660, 72618, 72646, 140445, 277491, 226648, 57740, 128551, 140324, 198659, 166581, 156560, 282964, 225124, 277491, 128707]
Number of articles read by user 7723: 39
Remaining articles after removing articles read by user 7723: 364009
Number of recommended articles that have already been read by user 7723: 0
recommended articles: [124988, 139886, 129016, 125516, 140603, 125546, 141291, 140328, 140815, 127495]


### 3. Collaborative-based Filtering

Le **Collaborative-based Filtering** est une méthode de recommandation qui se base sur les comportements passés des utilisateurs pour faire des prédictions sur ce qu'un utilisateur pourrait aimer.

**Principe**
L'idée principale est que si deux utilisateurs ont eu des comportements similaires par le passé (par exemple, ils ont aimé les mêmes films ou acheté les mêmes produits), alors ils sont susceptibles d'avoir des intérêts similaires à l'avenir.

**Types de Filtrage Collaboratif**
Il existe deux types principaux de filtrage collaboratif :

1. **Filtrage Collaboratif Basé sur les Utilisateurs** : Cette méthode trouve des utilisateurs similaires à l'utilisateur cible et recommande des éléments que ces utilisateurs similaires ont aimés.

2. **Filtrage Collaboratif Basé sur les Éléments** : Cette méthode trouve des éléments similaires à ceux que l'utilisateur cible a aimés et recommande ces éléments similaires.

3. **Filtrage Collaboratif Basé sur un Modèle** : Cette méthode utilise des techniques de modélisation, comme la factorisation de matrices ou le clustering, pour prédire l'intérêt d'un utilisateur pour un élément. Elle se base sur les comportements passés de tous les utilisateurs, ainsi que sur les évaluations que l'utilisateur cible a données à d'autres éléments.

**Calcul de la similarité**
La similarité entre les utilisateurs ou les éléments est généralement calculée en utilisant des techniques telles que la corrélation de Pearson ou la similarité cosinus.

**Note importante**
Contrairement au filtrage basé sur le contenu, le filtrage collaboratif ne nécessite pas d'informations détaillées sur les éléments. Il se base uniquement sur les interactions passées entre les utilisateurs et les éléments.

#### 3.1 Collaborative Filtering User Based

### Étapes du Code 

1. **Préparation des Données** :
    - Les colonnes `user_id` et `click_article_id` sont converties en type entier.
    - Les articles lus par l'utilisateur cible sont identifiés.

2. **Filtrage Collaboratif** :
    - Les utilisateurs qui ont lu les mêmes articles que l'utilisateur cible sont identifiés.
    - Les articles lus par ces utilisateurs similaires sont identifiés.
    - Les articles déjà lus par l'utilisateur cible sont retirés de ces recommandations.

3. **Vérification des Résultats** :
    - Si le nombre de recommandations est inférieur à n, les articles les plus populaires sont ajoutés aux recommandations jusqu'à atteindre n.
    - Les n premières recommandations sont retournées.

In [17]:
def collaborativeFilteringRecommendArticle(clicks, user_id, n=5):
    # Convert user_id and click_article_id to integer type
    clicks['user_id'] = clicks['user_id'].astype(int)
    clicks['click_article_id'] = clicks['click_article_id'].astype(int)
    
    # Get the articles read by the target user
    articles_read = clicks[clicks['user_id'] == user_id]['click_article_id'].values
    
    # Get the users who have read the same articles as the target user
    similar_users = clicks[clicks['click_article_id'].isin(articles_read)]['user_id'].unique()
    
    # Get the articles read by the similar users
    similar_users_articles = clicks[clicks['user_id'].isin(similar_users)]['click_article_id'].unique()
    
    # Remove the articles already read by the target user
    recommendations = [article for article in similar_users_articles if article not in articles_read]
    
    # If there are not enough recommendations, add the most popular articles
    if len(recommendations) < n:
        most_popular_articles = clicks['click_article_id'].value_counts().index.tolist()
        for article in most_popular_articles:
            if article not in recommendations and article not in articles_read:
                recommendations.append(article)
            if len(recommendations) == n:
                break
    
    return recommendations[:n]

In [18]:
collaborativeFilteringRecommendArticle(clicks_df,10)

[96210, 158094, 336245, 159197, 337441]

#### 3.2 Collaborative Filtering Model Based

`GridSearchCV` est une méthode de recherche exhaustive qui parcourt toutes les combinaisons possibles de paramètres pour trouver celle qui produit le meilleur score de validation croisée.

`GridSearchCV` fonctionne en entraînant et en évaluant un modèle pour chaque combinaison de paramètres. Il utilise la validation croisée pour évaluer la performance du modèle, ce qui signifie qu'il divise les données en un ensemble d'entraînement et un ensemble de test, entraîne le modèle sur l'ensemble d'entraînement, puis évalue la performance sur l'ensemble de test.

Une fois que `GridSearchCV` a terminé la recherche, nous pouvons obtenir les meilleurs paramètres en utilisant l'attribut `best_params_`. Nous pouvons ensuite utiliser ces paramètres pour entraîner notre modèle final.

#### 3.1 GridSearchCV

1. **Création de la Colonne 'click_count'** :
    Nous ajoutons une colonne click_count au DataFrame clicks_df qui compte le nombre de clics pour chaque article par utilisateur.

2. **Chargement d'une Fraction des Données** :
    Nous chargeons une fraction (10%) des données dans un dataset Surprise. Cela est fait en utilisant les colonnes user_id, click_article_id et click_count.

3. **Définition des Modèles et Paramètres** :
    Nous définissons différents modèles de prédiction ainsi que leurs grilles de paramètres respectifs pour la recherche de la meilleure configuration.

4. **Recherche des Meilleurs Paramètres** :
    Pour chaque modèle, nous utilisons GridSearchCV pour trouver les meilleurs paramètres en termes de RMSE et MAE à travers une validation croisée à 3 plis.

5. **Sélection du Meilleur Modèle** :
    Nous comparons les scores RMSE obtenus par chaque modèle et conservons le modèle avec le meilleur score ainsi que ses paramètres.

6. **Affichage des Résultats** :
    Nous affichons les meilleurs paramètres et les scores (RMSE et MAE) pour chaque modèle, ainsi que le meilleur modèle global.



In [19]:
from surprise import SVD, SVDpp, KNNWithMeans, CoClustering, SlopeOne, NormalPredictor
from surprise.model_selection import GridSearchCV

# Create a 'click_count' column
clicks_df['click_count'] = clicks_df.groupby(['user_id', 'click_article_id'])['click_timestamp'].transform('count')

# Load a fraction of the data into a Surprise dataset
reader = Reader(rating_scale=(0, clicks_df.click_count.max()))
data = Dataset.load_from_df(clicks_df[['user_id', 'click_article_id', 'click_count']].sample(frac=0.1, random_state=42), reader)


models = {
    SVD: {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]},
    SVDpp: {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]},
    KNNWithMeans: {'k': [20], 'sim_options': {'name': ['msd', 'cosine'], 'user_based': [False]}},
    CoClustering: {'n_cltr_u': [3, 5], 'n_cltr_i': [3, 5]},
    SlopeOne: {},
    NormalPredictor: {}
}

best_score = float('inf')
best_model = None
best_params = None

for model, param_grid in models.items():
    gs = GridSearchCV(model, param_grid, measures=['rmse', 'mae'], cv=3)
    gs.fit(data)
    params = gs.best_params['rmse']
    score = gs.best_score['rmse']
    print(f"Best parameters for {model.__name__}: {params}")
    print(f"Best RMSE for {model.__name__}: {score}")
    print(f"Best MAE for {model.__name__}: {gs.best_score['mae']}")
    if score < best_score:
        best_score = score
        best_model = model.__name__
        best_params = params

print(f"\nBest model: {best_model}")
print(f"Best parameters: {best_params}")
print(f"Best RMSE: {best_score}")

Best parameters for SVD: {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
Best RMSE for SVD: 0.23938850511580548
Best MAE for SVD: 0.056271097415754666
Best parameters for SVDpp: {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
Best RMSE for SVDpp: 0.23561784625022433
Best MAE for SVDpp: 0.05295362785914575
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Best parameters for KNNWithMeans: {'k': 20, 'sim_options': {'name': 'msd', 'user_based': False}}
Best RMSE for KNNWithMeans: 0.24423153346860757
Best MAE for KNNWithMeans: 0.06371725768887639
Best parameters for CoClustering: {'n_cltr_u': 3, 

#### 3.2 SVD
La décomposition en valeurs singulières (SVD, de l'anglais "Singular Value Decomposition") est une technique utilisée en réduction de dimension et en apprentissage automatique, notamment pour les systèmes de recommandation. Elle décompose une matrice en trois autres matrices, permettant ainsi de capturer les relations latentes entre les utilisateurs et les articles. En réduisant les dimensions, SVD aide à prédire les évaluations manquantes et à faire des recommandations plus précises.

1. **Création d'une Colonne 'click_count'** :
Tout d'abord, nous ajoutons une nouvelle colonne appelée click_count au DataFrame clicks_df. Cette colonne compte le nombre de clics que chaque utilisateur a effectués sur chaque article. Pour cela, nous utilisons la méthode groupby pour regrouper les données par user_id et click_article_id, puis nous appliquons la fonction transform pour calculer le nombre de clics.

2. **Chargement des Données** :
Ensuite, nous chargeons les données dans un dataset compatible avec la bibliothèque Surprise. Nous définissons une plage de notation allant de 0 au nombre maximum de clics observé dans click_count. Nous sélectionnons ensuite les colonnes pertinentes (user_id, click_article_id, click_count) pour créer ce dataset.

3. **Définition de la Grille de Paramètres** :
Nous définissons une grille de paramètres pour le modèle SVD. Cette grille spécifie différentes valeurs possibles pour plusieurs hyperparamètres :
- n_factors : le nombre de facteurs latents à utiliser.
- n_epochs : le nombre d'époques d'entraînement.
- lr_all : le taux d'apprentissage.
- reg_all : les paramètres de régularisation.

4. **Recherche en Grille avec Validation Croisée** :
Nous utilisons la méthode GridSearchCV de Surprise pour effectuer une recherche en grille des meilleurs paramètres. Cette méthode évalue les différentes combinaisons d'hyperparamètres en utilisant une validation croisée à 3 plis et en mesurant les performances avec deux métriques : RMSE (Root Mean Square Error) et MAE (Mean Absolute Error).

5. **Obtention et Affichage des Meilleurs Paramètres** :
Une fois la recherche en grille terminée, nous récupérons les meilleurs paramètres trouvés en termes de RMSE. Nous affichons ensuite ces paramètres ainsi que les scores de RMSE et MAE obtenus avec ces paramètres optimaux.

In [20]:
# Create a 'click_count' column
clicks_df['click_count'] = clicks_df.groupby(['user_id', 'click_article_id'])['click_timestamp'].transform('count')

# Load a fraction of the data into a Surprise dataset
reader = Reader(rating_scale=(0, clicks_df.click_count.max()))
data = Dataset.load_from_df(clicks_df[['user_id', 'click_article_id', 'click_count']], reader)

# Define the parameter grid
param_grid = {
    'n_factors': [10, 20, 30],
    'n_epochs': [5, 10],
    'lr_all': [0.001, 0.002, 0.003],
    'reg_all': [0.01, 0.02]
}

# Run a grid search with cross-validation
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Get the best parameters
best_params = gs.best_params['rmse']

print(f"Best parameters: {best_params}")

print(f"Best RMSE: {gs.best_score['rmse']}")
print(f"Best MAE: {gs.best_score['mae']}")

Best parameters: {'n_factors': 30, 'n_epochs': 10, 'lr_all': 0.003, 'reg_all': 0.01}
Best RMSE: 0.18807730866826675
Best MAE: 0.04740245811879407


In [21]:
# Train the model with the best parameters
model = SVD(**best_params)
trainset = data.build_full_trainset()
model.fit(trainset)

In [22]:
# Save the model
filename = '../model/finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

#### 3.3 Recommendations

##### 3.3.1 Définition de Précision@k et Rappel@k

##### Précision@k

La **précision@k** est une mesure qui évalue la proportion des éléments recommandés parmi les k premiers qui sont pertinents. En d'autres termes, elle indique la qualité des recommandations en termes de pertinence des k premières suggestions. Dans ce code, la précision@k est calculée comme suit :

1. On trie les prédictions par valeur estimée décroissante pour chaque utilisateur.
2. On prend les k premières prédictions.
3. On calcule le nombre d'éléments recommandés et pertinents parmi ces k premiers (`n_rel_and_rec_k`).
4. La précision@k est la proportion de ces éléments pertinents par rapport au nombre total d'éléments recommandés dans les k premiers (`n_rec_k`).

La formule est donc :

Précision@k = (Nombre d'éléments pertinents parmi les k premiers) / (Nombre total d'éléments recommandés dans les k premiers)

##### Rappel@k

Le **rappel@k** est une mesure qui évalue la proportion des éléments pertinents qui sont recommandés parmi les k premiers. En d'autres termes, elle indique la couverture des recommandations en termes de pertinence. Dans ce code, le rappel@k est calculé comme suit :

1. On trie les prédictions par valeur estimée décroissante pour chaque utilisateur.
2. On prend les k premières prédictions.
3. On calcule le nombre d'éléments recommandés et pertinents parmi ces k premiers (`n_rel_and_rec_k`).
4. On calcule le nombre total d'éléments pertinents pour l'utilisateur (`n_rel`).
5. Le rappel@k est la proportion des éléments pertinents recommandés parmi les k premiers par rapport au nombre total d'éléments pertinents.

La formule est donc :

Rappel@k = (Nombre d'éléments pertinents parmi les k premiers) / (Nombre total d'éléments pertinents)

#### En synthèse

- **Précision@k** mesure la qualité des recommandations parmi les k premiers éléments recommandés.
- **Rappel@k** mesure la couverture des éléments pertinents parmi les k premiers éléments recommandés.


In [23]:
def precision_recall_at_k(predictions, k_list=[5, 10]):
    '''Return precision and recall at k over all users for multiple values of k'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = {k: dict() for k in k_list}
    recalls = {k: dict() for k in k_list}
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        for k in k_list:
            # Number of recommended items in top k
            n_rec_k = len(user_ratings[:k])

            # Number of relevant and recommended items in top k
            n_rel_and_rec_k = sum((true_r == est) for (est, true_r) in user_ratings[:k])

            # Precision@K: Proportion of recommended items that are relevant
            precisions[k][uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

            # Number of relevant items
            n_rel = sum((true_r == est) for (est, true_r) in user_ratings)

            # Recall@K: Proportion of relevant items that are recommended
            recalls[k][uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [24]:
def collaborativeFilteringRecommendArticle(articles, clicks, user_id, n=5):
    # Convert user_id and click_article_id to integer type
    clicks['user_id'] = clicks['user_id'].astype(int)
    clicks['click_article_id'] = clicks['click_article_id'].astype(int)
    articles.index = articles.index.astype(int)
    
    # Check if user_id is in clicks
    if user_id not in clicks['user_id'].values:
        return f"Error: User ID {user_id} not found in clicks data."

    # Create a new DataFrame that counts the number of times a user clicked on an article
    click_counts = clicks.groupby(['user_id', 'click_article_id']).size().reset_index(name='click_count')

    # Use a smaller subset of data for the collaborative filtering to avoid memory issues
    data_subset = click_counts

    # Create a reader and a data object
    reader = Reader(rating_scale=(1, data_subset.click_count.max()))  # assuming a click count of at least 1
    data = Dataset.load_from_df(data_subset, reader)

    # Split the data into train and test sets
    trainset, testset = train_test_split(data, test_size=0.2)

    # Train a SVD model with the best parameters
    algo = SVD(n_factors=best_params['n_factors'],n_epochs=best_params['n_epochs'], lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])
    algo.fit(trainset)

    # Save the trained model to a pickle file
    with open('../model/trained_model.pkl', 'wb') as f:
        pickle.dump(algo, f)
    
    # Predict ratings for the testset
    predictions_test = algo.test(testset)

    # Calculate precision and recall at k
    precisions, recalls = precision_recall_at_k(predictions_test, k_list=[5, 10])
    for k in [5, 10]:
        avg_precision = sum(prec for prec in precisions[k].values()) / len(precisions[k])
        avg_recall = sum(rec for rec in recalls[k].values()) / len(recalls[k])
        print(f"Average Precision at {k}: {avg_precision}")
        print(f"Average Recall at {k}: {avg_recall}")

    # Get the list of articles read by the user
    articles_read = clicks[clicks['user_id'] == user_id]['click_article_id'].tolist()

    # Get the list of all articles
    all_articles = list(articles.index)

    # Remove the articles already read by the user
    articles_to_predict = [article for article in all_articles if article not in articles_read]

    # Get the predicted ratings for the articles not yet read by the user
    predictions = {article: algo.predict(user_id, article).est for article in articles_to_predict}

    # Get the top n articles
    top_n_articles = nlargest(n, predictions, key=predictions.get)

    return top_n_articles

In [25]:
recommended_articles = collaborativeFilteringRecommendArticle(embeddings_df_pca, clicks_df, 7723, n=10)

Average Precision at 5: 0.37128164921462264
Average Recall at 5: 0.9291109612466487
Average Precision at 10: 0.38862830158889294
Average Recall at 10: 0.9818411854381776


In [26]:
print("Recommended articles for user 7723:")
print(recommended_articles)

Recommended articles for user 7723:
[237071, 363925, 68851, 38823, 69463, 96173, 284209, 294899, 73431, 74396]


In [27]:
recommended_articles = collaborativeFilteringRecommendArticle(embeddings_df_pca, clicks_df, 20 , n=10)

Average Precision at 5: 0.36876561207265307
Average Recall at 5: 0.929712230405752
Average Precision at 10: 0.3858123828232042
Average Recall at 10: 0.9818147578353784


In [28]:
recommended_articles = collaborativeFilteringRecommendArticle(embeddings_df_pca, clicks_df, 5890 , n=10)

Average Precision at 5: 0.3722328751005706
Average Recall at 5: 0.9289775817281876
Average Precision at 10: 0.3895908904791733
Average Recall at 10: 0.9817093854315823


### 4. Hybrid based filtering

Le **filtrage hybride** est une méthode de recommandation qui combine plusieurs approches de filtrage pour améliorer la précision et la diversité des recommandations. Les approches couramment combinées incluent le filtrage collaboratif basé sur les utilisateurs, le filtrage collaboratif basé sur les articles et le filtrage basé sur le contenu. En utilisant plusieurs sources d'information, les systèmes hybrides peuvent compenser les faiblesses des méthodes individuelles et offrir des recommandations plus robustes.





#### 4.1 Hybrid based - User-based collaborative filtering and content-based filtering

### Étapes du Code du Modèle Hybride

1. **Préparation des Données** :
    - Les colonnes `user_id` et `click_article_id` sont converties en type entier.
    - Les articles lus par l'utilisateur cible sont identifiés.
    - Les embeddings des articles lus par l'utilisateur sont obtenus.
    - Les articles lus par l'utilisateur sont retirés de la liste des articles.

2. **Filtrage Basé sur le Contenu** :
    - La similarité cosinus entre les articles lus par l'utilisateur et les autres articles est calculée.
    - Les articles les plus similaires aux articles lus par l'utilisateur sont recommandés.

3. **Filtrage Collaboratif** :
    - Les utilisateurs qui ont lu les mêmes articles que l'utilisateur cible sont identifiés.
    - Les articles lus par ces utilisateurs similaires sont identifiés.
    - Les articles déjà lus par l'utilisateur cible sont retirés de ces recommandations.

4. **Combinaison des Résultats** :
    - Les recommandations du filtrage basé sur le contenu et du filtrage collaboratif sont combinées.
    - Si le nombre de recommandations est inférieur à n, les articles les plus populaires sont ajoutés aux recommandations jusqu'à atteindre n.
    - Les n premières recommandations sont retournées.

In [29]:
def hybrid_recommend_articles(articles, clicks, user_id, n=5):
    # Convert user_id and click_article_id to integer type
    clicks['user_id'] = clicks['user_id'].astype(int)
    clicks['click_article_id'] = clicks['click_article_id'].astype(int)
    articles.index = articles.index.astype(int)
    
    # Get the articles read by the target user
    articles_read = clicks[clicks['user_id'] == user_id]['click_article_id'].values
    
    # Get the embeddings of the articles read by the user
    articles_read_embedding = articles.loc[articles_read]
    
    # Remove the articles read by the user from the list of articles
    articles = articles.drop(articles_read)
    
    # Calculate the cosine similarity between the articles read by the user and the other articles
    matrix = cosine_similarity(articles_read_embedding, articles)
    
    # Recommend the articles most similar to the articles read by the user
    content_based_recommendations = [int(articles.index[np.argmax(row)]) for row in matrix]
    
    # Get the users who have read the same articles as the target user
    similar_users = clicks[clicks['click_article_id'].isin(articles_read)]['user_id'].unique()
    
    # Get the articles read by the similar users
    similar_users_articles = clicks[clicks['user_id'].isin(similar_users)]['click_article_id'].unique()
    
    # Remove the articles already read by the target user
    collaborative_filtering_recommendations = [article for article in similar_users_articles if article not in articles_read]
    
    # Combine the recommendations from the content-based and collaborative filtering approaches
    recommendations = content_based_recommendations + collaborative_filtering_recommendations
    
    # If there are not enough recommendations, add the most popular articles
    if len(recommendations) < n:
        most_popular_articles = clicks['click_article_id'].value_counts().index.tolist()
        for article in most_popular_articles:
            if article not in recommendations and article not in articles_read:
                recommendations.append(article)
            if len(recommendations) == n:
                break
    
    return recommendations[:n]

In [30]:
hybrid_recommend_articles(embeddings_df_pca, clicks_df, 10)

[193444, 194822, 100068, 69693, 34955]

#### 4.2 Hybrid based - Model-based collaborative filtering and content-based filtering

### Étapes du Code hybrid model based

1. **Préparation des Données** :
    - Convertir les colonnes `user_id` et `click_article_id` en type entier.
    - Vérifier si `user_id` est présent dans les données de clics.
    - Créer un nouveau DataFrame comptant le nombre de fois qu'un utilisateur a cliqué sur un article.

2. **Chargement et Division des Données** :
    - Utiliser un sous-ensemble des données pour le filtrage collaboratif afin d'éviter les problèmes de mémoire.
    - Créer un lecteur et un objet de données avec une échelle de notation basée sur le nombre de clics.
    - Diviser les données en ensembles d'entraînement et de test.

3. **Entraînement du Modèle** :
    - Entraîner un modèle SVD avec les meilleurs paramètres obtenus.
    - Prédire les notations pour l'ensemble de test.

4. **Calcul des Précisions et Rappels à k** :
    - Utiliser les fonctions de précision et de rappel pour évaluer les performances du modèle sur les valeurs k = 5 et k = 10.
    - Calculer et afficher les précisions et rappels moyens pour ces valeurs de k.

5. **Recommandation Basée sur le Filtrage Collaboratif** :
    - Obtenir la liste des articles lus par l'utilisateur.
    - Obtenir la liste de tous les articles disponibles.
    - Exclure les articles déjà lus par l'utilisateur.
    - Prédire les notations pour les articles non lus et obtenir les n meilleurs articles.

6. **Filtrage Basé sur le Contenu** :
    - Utiliser une fonction de recommandation basée sur le contenu (`recommend_articles`) pour obtenir les n meilleurs articles.

7. **Combinaison des Résultats** :
    - Combiner les résultats des filtres collaboratifs et basés sur le contenu pour obtenir les meilleures recommandations.

In [31]:
def hybridRecommendArticle(articles, clicks, user_id, n=5):
    # Convert user_id and click_article_id to integer type
    clicks['user_id'] = clicks['user_id'].astype(int)
    clicks['click_article_id'] = clicks['click_article_id'].astype(int)
    articles.index = articles.index.astype(int)
    
    # Check if user_id is in clicks
    if user_id not in clicks['user_id'].values:
        return f"Error: User ID {user_id} not found in clicks data."

    # Create a new DataFrame that counts the number of times a user clicked on an article
    click_counts = clicks.groupby(['user_id', 'click_article_id']).size().reset_index(name='click_count')

    # Use a smaller subset of data for the collaborative filtering to avoid memory issues
    data_subset = click_counts

    # Create a reader and a data object
    reader = Reader(rating_scale=(1, data_subset.click_count.max()))  # assuming a click count of at least 1
    data = Dataset.load_from_df(data_subset, reader)

    # Split the data into train and test sets
    trainset, testset = train_test_split(data, test_size=0.2)

    # Train a SVD model with the best parameters
    algo = SVD(n_factors=best_params['n_factors'],n_epochs=best_params['n_epochs'], lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])
    algo.fit(trainset)

    # Save the trained model
    #with open(model_path + 'model.pkl', 'wb') as f:
        #pickle.dump(algo, f)

    # Predict ratings for the testset
    predictions_test = algo.test(testset)

    # Calculate precision and recall at k
    precisions, recalls = precision_recall_at_k(predictions_test, k_list=[5, 10])
    for k in [5, 10]:
        avg_precision = sum(prec for prec in precisions[k].values()) / len(precisions[k])
        avg_recall = sum(rec for rec in recalls[k].values()) / len(recalls[k])
        print(f"Average Precision at {k}: {avg_precision}")
        print(f"Average Recall at {k}: {avg_recall}")

    # Get the list of articles read by the user
    articles_read = clicks[clicks['user_id'] == user_id]['click_article_id'].tolist()

    # Get the list of all articles
    all_articles = list(articles.index)

    # Remove the articles already read by the user
    articles_to_predict = [article for article in all_articles if article not in articles_read]

    # Get the predicted ratings for the articles not yet read by the user
    predictions = {article: algo.predict(user_id, article).est for article in articles_to_predict}

    # Get the top n articles
    top_n_articles_collab = nlargest(n, predictions, key=predictions.get)

    # Content-based filtering
    top_n_articles_content = recommend_articles(articles, clicks, user_id, n)

    # Combine the results of collaborative and content-based filtering
    top_n_articles = top_n_articles_collab + top_n_articles_content

    return top_n_articles

In [32]:
recommended_articles = hybridRecommendArticle(embeddings_df_pca, clicks_df, 7723, n=10)

Average Precision at 5: 0.3687001003359955
Average Recall at 5: 0.9292172292308205
Average Precision at 10: 0.38611761542141
Average Recall at 10: 0.9821256779209091
Articles read by user 7723: [214455, 9308, 9649, 129799, 141548, 336220, 353673, 337192, 84763, 107179, 199197, 271400, 84835, 338339, 60252, 303565, 31520, 36685, 36609, 163505, 123434, 141050, 313504, 272660, 72618, 72646, 140445, 277491, 226648, 57740, 128551, 140324, 198659, 166581, 156560, 282964, 225124, 277491, 128707]
Number of articles read by user 7723: 39
Remaining articles after removing articles read by user 7723: 364009
Number of recommended articles that have already been read by user 7723: 0


In [33]:
recommended_articles = hybridRecommendArticle(embeddings_df_pca, clicks_df, 20, n=10)

Average Precision at 5: 0.375531967466616
Average Recall at 5: 0.9286110447763688
Average Precision at 10: 0.3929754116206176
Average Recall at 10: 0.9818623142331041
Articles read by user 20: [157541, 157485]
Number of articles read by user 20: 2
Remaining articles after removing articles read by user 20: 364045
Number of recommended articles that have already been read by user 20: 0


In [34]:
recommended_articles = hybridRecommendArticle(embeddings_df_pca, clicks_df, 5890, n=10)

Average Precision at 5: 0.3713205647566355
Average Recall at 5: 0.9286830976257195
Average Precision at 10: 0.3888101077521943
Average Recall at 10: 0.98184394435023
Articles read by user 5890: [207757, 21267, 264111, 144930, 66066, 111933, 2344, 236189, 3828, 132820, 142148, 66346, 111482, 198222, 283976, 114458, 207309, 288840, 202455, 206934, 59452, 194573, 207622, 65495, 65413, 288564, 183615, 283608, 9175, 206168, 206639, 47944, 206805, 217579, 162147, 140299, 119541, 146110, 225697, 96120, 194599, 145823, 96161, 177817, 235440, 206808, 120875, 36622, 65656, 14324, 270326, 59408, 65407, 206534, 29994, 133142, 76373, 208199, 16882, 47862, 206730, 61749, 119455, 96106, 95716, 96269, 166462, 233717, 199197, 84136, 66066, 198538, 207757, 21267, 145165, 29975, 65991, 9668, 198758, 59219, 57550, 261791, 264111, 57757, 118918, 225010, 144930, 285675, 261792, 183237, 111933, 208094, 242023, 57576, 58043, 235287, 59225, 206778, 58445, 360990, 293467, 244858, 76257, 58733, 136208, 97608, 22